<a href="https://colab.research.google.com/github/EkaFitriRamadani/PengantarDeepLearning2022/blob/main/DeepLearningComputationD2l_12March22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Layers and Blocks

In [1]:
import tensorflow as tf

net = tf.keras.models.Sequential([
    tf.keras.layers.Dense(256, activation=tf.nn.relu),
    tf.keras.layers.Dense(10),
])

X = tf.random.uniform((2, 20))
net(X)

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[-0.06586166,  0.16533422,  0.04252891,  0.31770214,  0.13674313,
         0.08272987, -0.03047456,  0.5518371 , -0.02851011,  0.02550869],
       [-0.24300934,  0.06520978,  0.17154104,  0.27869678, -0.01986345,
        -0.14418414, -0.08837392,  0.46821177, -0.05771974, -0.01701859]],
      dtype=float32)>

## 1.1 Custom Block

In [2]:
class MLP(tf.keras.Model):
    # Declare a layer with model parameters. Here, we declare two fully
    # connected layers
    def __init__(self):
        # Call the constructor of the `MLP` parent class `Model` to perform
        # the necessary initialization. In this way, other function arguments
        # can also be specified during class instantiation, such as the model
        # parameters, `params` (to be described later)
        super().__init__()
        # Hidden layer
        self.hidden = tf.keras.layers.Dense(units=256, activation=tf.nn.relu)
        self.out = tf.keras.layers.Dense(units=10)  # Output layer

    # Define the forward propagation of the model, that is, how to return the
    # required model output based on the input `X`
    def call(self, X):
        return self.out(self.hidden((X)))

In [3]:
net = MLP()
net(X)

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[ 0.17223176,  0.05949853, -0.21089816, -0.04591832,  0.45338985,
         0.11274566, -0.2087471 , -0.08370548,  0.06432437, -0.23954317],
       [-0.01028988,  0.1250643 , -0.38192153, -0.00952755,  0.3627178 ,
         0.10472718, -0.06191794, -0.17313616,  0.03693228, -0.29428396]],
      dtype=float32)>

## 1.2 The Sequential Block

In [4]:
class MySequential(tf.keras.Model):
    def __init__(self, *args):
        super().__init__()
        self.modules = []
        for block in args:
            # Here, `block` is an instance of a `tf.keras.layers.Layer`
            # subclass
            self.modules.append(block)

    def call(self, X):
        for module in self.modules:
            X = module(X)
        return X

In [5]:
net = MySequential(
    tf.keras.layers.Dense(units=256, activation=tf.nn.relu),
    tf.keras.layers.Dense(10))
net(X)

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[ 0.47536388, -0.21874963, -0.2713691 , -0.41362306,  0.06972843,
        -0.16278227,  0.04355606, -0.07421583,  0.17058577, -0.44836605],
       [ 0.19394462,  0.35623017,  0.11589262, -0.18036075,  0.08135638,
        -0.14599262, -0.16809246, -0.44781402,  0.08764486, -0.44863263]],
      dtype=float32)>

## 1.3 Executing Code in The Forward Propagation Function

In [6]:
class FixedHiddenMLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.flatten = tf.keras.layers.Flatten()
        # Random weight parameters created with `tf.constant` are not updated
        # during training (i.e., constant parameters)
        self.rand_weight = tf.constant(tf.random.uniform((20, 20)))
        self.dense = tf.keras.layers.Dense(20, activation=tf.nn.relu)

    def call(self, inputs):
        X = self.flatten(inputs)
        # Use the created constant parameters, as well as the `relu` and
        # `matmul` functions
        X = tf.nn.relu(tf.matmul(X, self.rand_weight) + 1)
        # Reuse the fully-connected layer. This is equivalent to sharing
        # parameters with two fully-connected layers
        X = self.dense(X)
        # Control flow
        while tf.reduce_sum(tf.math.abs(X)) > 1:
            X /= 2
        return tf.reduce_sum(X)

In [7]:
net = FixedHiddenMLP()
net(X)

<tf.Tensor: shape=(), dtype=float32, numpy=0.55767643>

In [8]:
# We can mix and match various ways of assembling blocks together. In the following example, we nest blocks in some creative ways.
class NestMLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.net = tf.keras.Sequential()
        self.net.add(tf.keras.layers.Dense(64, activation=tf.nn.relu))
        self.net.add(tf.keras.layers.Dense(32, activation=tf.nn.relu))
        self.dense = tf.keras.layers.Dense(16, activation=tf.nn.relu)

    def call(self, inputs):
        return self.dense(self.net(inputs))

chimera = tf.keras.Sequential()
chimera.add(NestMLP())
chimera.add(tf.keras.layers.Dense(20))
chimera.add(FixedHiddenMLP())
chimera(X)

<tf.Tensor: shape=(), dtype=float32, numpy=0.7103485>

# 2. Parameter Management

In [9]:
import tensorflow as tf

net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(4, activation=tf.nn.relu),
    tf.keras.layers.Dense(1),
])

X = tf.random.uniform((2, 4))
net(X)

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[0.15245457],
       [0.7179609 ]], dtype=float32)>

## 2.1 Parameter Access

In [10]:
print(net.layers[2].weights)

[<tf.Variable 'dense_13/kernel:0' shape=(4, 1) dtype=float32, numpy=
array([[ 0.6833309 ],
       [-0.4041291 ],
       [ 0.24136364],
       [ 0.9802327 ]], dtype=float32)>, <tf.Variable 'dense_13/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>]


### 2.1.1 Targeted parameters

In [11]:
print(type(net.layers[2].weights[1]))
print(net.layers[2].weights[1])
print(tf.convert_to_tensor(net.layers[2].weights[1]))

<class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'>
<tf.Variable 'dense_13/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>
tf.Tensor([0.], shape=(1,), dtype=float32)


2.1.2 All Parameters at Once

In [12]:
print(net.layers[1].weights)
print(net.get_weights())

[<tf.Variable 'dense_12/kernel:0' shape=(4, 4) dtype=float32, numpy=
array([[ 0.04900992, -0.18677682,  0.53723747,  0.56810826],
       [ 0.17433935, -0.44567072, -0.54528   , -0.49077946],
       [ 0.56810266, -0.5234345 , -0.42286602,  0.6285024 ],
       [-0.4627224 , -0.11166126, -0.11746407, -0.5467074 ]],
      dtype=float32)>, <tf.Variable 'dense_12/bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>]
[array([[ 0.04900992, -0.18677682,  0.53723747,  0.56810826],
       [ 0.17433935, -0.44567072, -0.54528   , -0.49077946],
       [ 0.56810266, -0.5234345 , -0.42286602,  0.6285024 ],
       [-0.4627224 , -0.11166126, -0.11746407, -0.5467074 ]],
      dtype=float32), array([0., 0., 0., 0.], dtype=float32), array([[ 0.6833309 ],
       [-0.4041291 ],
       [ 0.24136364],
       [ 0.9802327 ]], dtype=float32), array([0.], dtype=float32)]


In [13]:
net.get_weights()[1]

array([0., 0., 0., 0.], dtype=float32)

### 2.1.3 Collecting Parameters from Nested Blocks

In [14]:
def block1(name):
    return tf.keras.Sequential([
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(4, activation=tf.nn.relu)],
        name=name)

def block2():
    net = tf.keras.Sequential()
    for i in range(4):
        # Nested here
        net.add(block1(name=f'block-{i}'))
    return net

rgnet = tf.keras.Sequential()
rgnet.add(block2())
rgnet.add(tf.keras.layers.Dense(1))
rgnet(X)

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[-0.11723108],
       [ 0.        ]], dtype=float32)>

In [15]:
print(rgnet.summary())

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_5 (Sequential)   (2, 4)                    80        
                                                                 
 dense_18 (Dense)            (2, 1)                    5         
                                                                 
Total params: 85
Trainable params: 85
Non-trainable params: 0
_________________________________________________________________
None


In [16]:
rgnet.layers[0].layers[1].layers[1].weights[1]

<tf.Variable 'dense_15/bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>

## 2.2 Parameters Initialization

### 2.2.1 Built-in Initialization

In [17]:
net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(
        4, activation=tf.nn.relu,
        kernel_initializer=tf.random_normal_initializer(mean=0, stddev=0.01),
        bias_initializer=tf.zeros_initializer()),
    tf.keras.layers.Dense(1)])

net(X)
net.weights[0], net.weights[1]

(<tf.Variable 'dense_19/kernel:0' shape=(4, 4) dtype=float32, numpy=
 array([[-0.00516976,  0.01643996,  0.00722332,  0.00189736],
        [ 0.00175535, -0.00054569, -0.01581811,  0.00384197],
        [-0.00106413, -0.01169838, -0.00712359,  0.00348602],
        [-0.0050912 , -0.00491398, -0.00405891, -0.00981443]],
       dtype=float32)>,
 <tf.Variable 'dense_19/bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>)

In [18]:
# We can also initialize all the parameters to a given constant value (say, 1).
net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(
        4, activation=tf.nn.relu,
        kernel_initializer=tf.keras.initializers.Constant(1),
        bias_initializer=tf.zeros_initializer()),
    tf.keras.layers.Dense(1),
])

net(X)
net.weights[0], net.weights[1]

(<tf.Variable 'dense_21/kernel:0' shape=(4, 4) dtype=float32, numpy=
 array([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]], dtype=float32)>,
 <tf.Variable 'dense_21/bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>)

In [19]:
# We can also apply different initializers for certain blocks. 
# For example, below we initialize the first layer with the Xavier 
# initializer and initialize the second layer to a constant value of 42.
net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(
        4,
        activation=tf.nn.relu,
        kernel_initializer=tf.keras.initializers.GlorotUniform()),
    tf.keras.layers.Dense(
        1, kernel_initializer=tf.keras.initializers.Constant(42)),
])

net(X)
print(net.layers[1].weights[0])
print(net.layers[2].weights[0])

<tf.Variable 'dense_23/kernel:0' shape=(4, 4) dtype=float32, numpy=
array([[ 0.4368561 , -0.05866247,  0.73839396,  0.3076182 ],
       [-0.36384845,  0.7552263 , -0.17514062,  0.7770278 ],
       [ 0.33943218,  0.8638367 ,  0.22952491, -0.82596356],
       [-0.6454965 , -0.01773757, -0.168917  ,  0.28436238]],
      dtype=float32)>
<tf.Variable 'dense_24/kernel:0' shape=(4, 1) dtype=float32, numpy=
array([[42.],
       [42.],
       [42.],
       [42.]], dtype=float32)>


### 2.2.2 Custom Initialization

In [20]:
class MyInit(tf.keras.initializers.Initializer):
    def __call__(self, shape, dtype=None):
        data=tf.random.uniform(shape, -10, 10, dtype=dtype)
        factor=(tf.abs(data) >= 5)
        factor=tf.cast(factor, tf.float32)
        return data * factor

net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(
        4,
        activation=tf.nn.relu,
        kernel_initializer=MyInit()),
    tf.keras.layers.Dense(1),
])

net(X)
print(net.layers[1].weights[0])

<tf.Variable 'dense_25/kernel:0' shape=(4, 4) dtype=float32, numpy=
array([[ 0.       , -0.       ,  7.826689 ,  0.       ],
       [ 0.       ,  0.       ,  7.310753 , -0.       ],
       [ 0.       , -0.       ,  6.3274117,  6.2730865],
       [ 8.548021 , -0.       ,  9.1618   ,  7.325056 ]], dtype=float32)>


In [21]:
net.layers[1].weights[0][:].assign(net.layers[1].weights[0] + 1)
net.layers[1].weights[0][0, 0].assign(42)
net.layers[1].weights[0]

<tf.Variable 'dense_25/kernel:0' shape=(4, 4) dtype=float32, numpy=
array([[42.       ,  1.       ,  8.826689 ,  1.       ],
       [ 1.       ,  1.       ,  8.310753 ,  1.       ],
       [ 1.       ,  1.       ,  7.3274117,  7.2730865],
       [ 9.548021 ,  1.       , 10.1618   ,  8.325056 ]], dtype=float32)>

## 2.3 Tied parameters

In [22]:
# tf.keras behaves a bit differently. It removes the duplicate layer
# automatically
shared = tf.keras.layers.Dense(4, activation=tf.nn.relu)
net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    shared,
    shared,
    tf.keras.layers.Dense(1),
])

net(X)
# Check whether the parameters are different
print(len(net.layers) == 3)

True


# 3. Deffered Initialization

## 3.1 Instantiating a Network

In [23]:
import tensorflow as tf

net = tf.keras.models.Sequential([
    tf.keras.layers.Dense(256, activation=tf.nn.relu),
    tf.keras.layers.Dense(10),
])

In [24]:
[net.layers[i].get_weights() for i in range(len(net.layers))]

[[], []]

In [25]:
X = tf.random.uniform((2, 20))
net(X)
[w.shape for w in net.get_weights()]

[(20, 256), (256,), (256, 10), (10,)]

# 4. Custom Layers

## 4.1 Layers without Parameters

In [26]:
import tensorflow as tf


class CenteredLayer(tf.keras.Model):
    def __init__(self):
        super().__init__()

    def call(self, inputs):
        return inputs - tf.reduce_mean(inputs)

In [27]:
layer = CenteredLayer()
layer(tf.constant([1, 2, 3, 4, 5]))

<tf.Tensor: shape=(5,), dtype=int32, numpy=array([-2, -1,  0,  1,  2], dtype=int32)>

In [28]:
# We can now incorporate our layer as a component in constructing more complex models.
net = tf.keras.Sequential([tf.keras.layers.Dense(128), CenteredLayer()])

In [29]:
Y = net(tf.random.uniform((4, 8)))
tf.reduce_mean(Y)

<tf.Tensor: shape=(), dtype=float32, numpy=-1.1641532e-09>

## 4.2 Layers with Parameters

In [30]:
class MyDense(tf.keras.Model):
    def __init__(self, units):
        super().__init__()
        self.units = units

    def build(self, X_shape):
        self.weight = self.add_weight(name='weight',
            shape=[X_shape[-1], self.units],
            initializer=tf.random_normal_initializer())
        self.bias = self.add_weight(
            name='bias', shape=[self.units],
            initializer=tf.zeros_initializer())

    def call(self, X):
        linear = tf.matmul(X, self.weight) + self.bias
        return tf.nn.relu(linear)

In [31]:
# Next, we instantiate the MyDense class and access its model parameters.
dense = MyDense(3)
dense(tf.random.uniform((2, 5)))
dense.get_weights()

[array([[ 0.01041123,  0.05840929,  0.02973732],
        [ 0.02792704,  0.07402581, -0.07510598],
        [-0.02465218, -0.06562633,  0.04169067],
        [-0.02896944, -0.00495695, -0.0598686 ],
        [-0.00354981,  0.02209506, -0.0930787 ]], dtype=float32),
 array([0., 0., 0.], dtype=float32)]

In [32]:
# We can directly carry out forward propagation calculations using custom layers.
dense(tf.random.uniform((2, 5)))

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[0.        , 0.05495436, 0.        ],
       [0.        , 0.08472928, 0.        ]], dtype=float32)>

In [33]:
# net = tf.keras.models.Sequential([MyDense(8), MyDense(1)])
net = tf.keras.models.Sequential([MyDense(8), MyDense(1)])
net(tf.random.uniform((2, 64)))

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[0.        ],
       [0.00074074]], dtype=float32)>

# 5. File I/O

## 5.1 Loading and Saving Tensors

In [34]:
import numpy as np
import tensorflow as tf

x = tf.range(4)
np.save('x-file.npy', x)

In [35]:
# We can now read the data from the stored file back into memory.
x2 = np.load('x-file.npy', allow_pickle=True)
x2

array([0, 1, 2, 3], dtype=int32)

In [36]:
# We can store a list of tensors and read them back into memory.
y = tf.zeros(4)
np.save('xy-files.npy', [x, y])
x2, y2 = np.load('xy-files.npy', allow_pickle=True)
(x2, y2)

(array([0., 1., 2., 3.]), array([0., 0., 0., 0.]))

In [37]:
# We can even write and read a dictionary that maps from strings to tensors. 
# This is convenient when we want to read or write all the weights in a model.
mydict = {'x': x, 'y': y}
np.save('mydict.npy', mydict)
mydict2 = np.load('mydict.npy', allow_pickle=True)
mydict2

array({'x': <tf.Tensor: shape=(4,), dtype=int32, numpy=array([0, 1, 2, 3], dtype=int32)>, 'y': <tf.Tensor: shape=(4,), dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>},
      dtype=object)

## 5.2 Loading and Saving Model Parameters

In [38]:
class MLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.flatten = tf.keras.layers.Flatten()
        self.hidden = tf.keras.layers.Dense(units=256, activation=tf.nn.relu)
        self.out = tf.keras.layers.Dense(units=10)

    def call(self, inputs):
        x = self.flatten(inputs)
        x = self.hidden(x)
        return self.out(x)

net = MLP()
X = tf.random.uniform((2, 20))
Y = net(X)

In [39]:
# Next, we store the parameters of the model as a file with the name “mlp.params”.
net.save_weights('mlp.params')

In [40]:
# To recover the model, we instantiate a clone of the original MLP model. 
# Instead of randomly initializing the model parameters, we read the parameters stored in the file directly.
clone = MLP()
clone.load_weights('mlp.params')

In [41]:
# Since both instances have the same model parameters, the computational
# result of the same input X should be the same. Let us verify this.
Y_clone = clone(X)
Y_clone == Y

<tf.Tensor: shape=(2, 10), dtype=bool, numpy=
array([[ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True]])>

# 6. GPUs

In [42]:
!nvidia-smi

Sat Mar 19 02:15:12 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   72C    P0    76W / 149W |    195MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## 6.1 Computing Devices

In [43]:
import tensorflow as tf

tf.device('/CPU:0'), tf.device('/GPU:0'), tf.device('/GPU:1')

(<tensorflow.python.eager.context._EagerDeviceContext at 0x7ff8b07f44b0>,
 <tensorflow.python.eager.context._EagerDeviceContext at 0x7ff8b07f4e10>)

In [44]:
# We can query the number of available GPUs.
len(tf.config.experimental.list_physical_devices('GPU'))

1

In [45]:
def try_gpu(i=0):
    """Return gpu(i) if exists, otherwise return cpu()."""
    if len(tf.config.experimental.list_physical_devices('GPU')) >= i + 1:
        return tf.device(f'/GPU:{i}')
    return tf.device('/CPU:0')

def try_all_gpus():
    """Return all available GPUs, or [cpu(),] if no GPU exists."""
    num_gpus = len(tf.config.experimental.list_physical_devices('GPU'))
    devices = [tf.device(f'/GPU:{i}') for i in range(num_gpus)]
    return devices if devices else [tf.device('/CPU:0')]

try_gpu(), try_gpu(10), try_all_gpus()

(<tensorflow.python.eager.context._EagerDeviceContext at 0x7ff8b07f8e60>,
 [<tensorflow.python.eager.context._EagerDeviceContext at 0x7ff8b07f5fa0>])

## 6.2 Tensors and GPUs

In [46]:
x = tf.constant([1, 2, 3])
x.device

'/job:localhost/replica:0/task:0/device:GPU:0'

### 6.2.1 Storage on the GPU

In [47]:
with try_gpu():
    X = tf.ones((2, 3))
X

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[1., 1., 1.],
       [1., 1., 1.]], dtype=float32)>

In [48]:
# Assuming that you have at least two GPUs, the following code will create a random tensor on the second GPU.
with try_gpu(1):
    Y = tf.random.uniform((2, 3))
Y

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[0.7587992 , 0.05003738, 0.9273449 ],
       [0.78470933, 0.2464801 , 0.1937598 ]], dtype=float32)>

### 6.2.2 Copying

In [49]:
with try_gpu(1):
    Z = X
print(X)
print(Z)

tf.Tensor(
[[1. 1. 1.]
 [1. 1. 1.]], shape=(2, 3), dtype=float32)
tf.Tensor(
[[1. 1. 1.]
 [1. 1. 1.]], shape=(2, 3), dtype=float32)


In [50]:
# Now that the data are on the same GPU (both Z and Y are), we can add them up.
Y + Z

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[1.7587992, 1.0500374, 1.9273449],
       [1.7847093, 1.2464801, 1.1937598]], dtype=float32)>

In [51]:
with try_gpu(1):
    Z2 = Z
Z2 is Z

True

## 6.3 Neural Networks and GPUs

In [52]:
strategy = tf.distribute.MirroredStrategy()
with strategy.scope():
    net = tf.keras.models.Sequential([
        tf.keras.layers.Dense(1)])

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


In [53]:
net(X)

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[0.57049465],
       [0.57049465]], dtype=float32)>

In [54]:
net.layers[0].weights[0].device, net.layers[0].weights[1].device

('/job:localhost/replica:0/task:0/device:GPU:0',
 '/job:localhost/replica:0/task:0/device:GPU:0')